In [57]:
import sys, os
from typing import DefaultDict

In [58]:

def parse_bw(filepath, target):
    if target < 0 or target > 5:
        print('Wrong target range!')
        sys.exit(1)
        
    """
    target
    0: msg_size
    1: iter
    2: bw_peak
    3: bw_avg
    4: msg_rate
    5: all
    """
    res = []
    
    f = open(filepath, "r")
    lines = f.readlines()
    
    startline = 0
    for j in range(len(lines)):
        if lines[j][0:4] == " #by":
            startline = j + 1
            break
    endline = startline
    while endline < len(lines):
        if lines[endline][1:4] == '===':
            break
        endline += 1
        
    curline = startline
    while curline < endline:
        if lines[curline][0:2] == ' --':
            break
        if lines[curline][0] != ' ':
            curline += 1
            continue
        line = lines[curline].split()
        try:            
            # (msg_size, iteration, bw_peak, bw_avg, msg_rate)
            if target == 5:
                row = []
                row.append(line[0])
                row.append(line[1])
                row.append(line[2])
                row.append(line[3])
                row.append(line[4])
                res.append(row)
            else:
                res.append(line[target])
        except:
            curline += 1
            continue
        curline += 1
    
    return res

def parse_lat(filepath):
    f = open(filepath, "r")
    lines = f.readlines()
    
    startline = 0
    for j in range(len(lines)):
        if lines[j][0:4] == " #by":
            startline = j + 1
            break
    endline = startline
    while endline < len(lines):
        if lines[endline][1:4] == '===':
            break
        endline += 1
        
    curline = startline
    while curline < endline:
        if lines[curline][0:2] == ' --':
            break
        if lines[curline][0] != ' ':
            curline += 1
            continue
        line = lines[curline].split()
        try:            
            return line
                
        except:
            curline += 1
            continue

In [59]:

# Output format = [[ data1 ], [data2], ... , columns = [] ]
dataset = []
columns = ['rnic', 'test_node', 'test_type', 'tx_depth', 'mtu', 'bg_msgsize', 'bg_bw', 'bg_msgrate', 'tar_msgsize', 'tar_bw', 'tar_msgrate', 'bw_sum']

###################################
## Modify Here
TARGET = 'test_result_X5_ETH'
OUTPUT = 'X5ETH_bw_comp.csv'
###################################

target_dir = './' + TARGET
if os.path.isdir(target_dir) == False:
    print("Directory is not valid!")
    sys.exit()
    
file_list = os.listdir(target_dir)

### Get target files ###
target_list = []
bw_t = ['rb', 'wb', 'sb']
for target in file_list:
    splt_target = target.split('_')
    if splt_target[2] in bw_t and splt_target[7] == 'tf':
        target_list.append(target)

for t in target_list:
    # print(t)
    row = []
    splt_t = t.split('_')
    
    # add testing env
    row.append(splt_t[0])
    row.append(splt_t[1])
    row.append(splt_t[2])
    row.append(splt_t[3][1:])
    row.append(splt_t[4][1:])
    
    # add bg_bw
    bg_data = parse_bw(target_dir + '/' + t[:-2] + 'bf', 5)
    choose = int(len(bg_data)-1)
    
    chosen_bg = bg_data[choose]
    row.append(chosen_bg[0]) # msg size
    row.append(chosen_bg[3]) # avg bw
    row.append(chosen_bg[4]) # msg rate
    
    # add tar_bw data
    tar_data = parse_bw(target_dir + '/' + t, 5)
    choose = int(len(tar_data)-1)
    
    chosen_tar = tar_data[choose]
    row.append(chosen_tar[0]) # msg size
    row.append(chosen_tar[3]) # avg bw
    row.append(chosen_tar[4]) # msg rate
    
    # add sum of bws
    row.append(str(float(chosen_bg[3]) + float(chosen_tar[3])))

    # print(row)
    dataset.append(row)
    # break

# print(dataset)

In [60]:
import pandas as pd

df = pd.DataFrame(dataset, columns=columns)

print(df)
df.to_csv(OUTPUT)

      rnic test_node test_type tx_depth   mtu  bg_msgsize    bg_bw bg_msgrate  \
0    X4ETH      Y303        rb      128  1024     1048576  2188.19   0.002188   
1    X4ETH      Y303        rb      128  1024     1048576  2188.79   0.002189   
2    X4ETH      Y303        rb      128  1024     1048576  2189.19   0.002189   
3    X4ETH      Y303        rb      128  1024     1048576  2188.99   0.002189   
4    X4ETH      Y303        rb      128  1024     1048576  2188.39   0.002188   
..     ...       ...       ...      ...   ...         ...      ...        ...   
955  X4ETH      Y303        wb        4   512  1073741824  2047.99   0.000002   
956  X4ETH      Y303        wb        4   512  1073741824  3071.99   0.000003   
957  X4ETH      Y303        wb        4   512  1073741824  2047.99   0.000002   
958  X4ETH      Y303        wb        4   512  1073741824  2047.97   0.000002   
959  X4ETH      Y303        wb        4   512  1073741824  2662.39   0.000003   

    tar_msgsize   tar_bw ta